---
sidebar_label: Box
---

"Box" 是 MUI 中

# BoxLoader 和 BoxBlobLoader

`langchain-box` 包提供了两种从 Box 索引文件的方法：`BoxLoader` 和 `BoxBlobLoader`。`BoxLoader` 允许您摄取在 Box 中具有文本表示的文件文本表示。`BoxBlobLoader` 允许您下载任何文档或图像文件的 blob，以便您选择的 blob 解析器进行处理。

本笔记本详细介绍了两者的入门。有关 BoxLoader 所有功能和配置的详细文档，请访问 [BoxLoader](https://python.langchain.com/api_reference/box/document_loaders/langchain_box.document_loaders.box.BoxLoader.html) 和 [BoxBlobLoader](https://python.langchain.com/api_reference/box/document_loaders/langchain_box.blob_loaders.box.BoxBlobLoader.html) 的 API 参考页面。

## 概述

`BoxLoader` 类可帮助您以 Langchain 的 `Document` 格式从 Box 获取非结构化内容。您可以使用包含 Box 文件 ID 的 `List[str]`，或者包含 Box 文件夹 ID 的 `str` 来实现此目的。

`BoxBlobLoader` 类可帮助您以 Langchain 的 `Blob` 格式从 Box 获取非结构化内容。您可以使用包含 Box 文件 ID 的 `List[str]`、包含 Box 文件夹 ID 的 `str`、搜索查询或 `BoxMetadataQuery` 来实现此目的。

如果通过文件夹 ID 从文件夹获取文件，您还可以设置一个 `Bool` 来告知加载器是否获取该文件夹中的所有子文件夹。

:::info
一个 Box 实例可以包含 PB 级的文件，文件夹也可以包含数百万个文件。在选择要索引的文件夹时请谨慎。我们建议永远不要递归地获取文件夹 0 中的所有文件。文件夹 ID 0 是您的根文件夹。
:::

`BoxLoader` 将会跳过没有文本表示的文件，而 `BoxBlobLoader` 将会为所有文档和图像文件返回 blob。

### 集成详情

| 类 | 包 | 本地 | 可序列化 | JS 支持 |
| :--- | :--- | :---: | :---: |  :---: |
| [BoxLoader](https://python.langchain.com/api_reference/box/document_loaders/langchain_box.document_loaders.box.BoxLoader.html) | [langchain_box](https://python.langchain.com/api_reference/box/index.html) | ✅ | ❌ | ❌ |
| [BoxBlobLoader](https://python.langchain.com/api_reference/box/document_loaders/langchain_box.blob_loaders.box.BoxBlobLoader.html) | [langchain_box](https://python.langchain.com/api_reference/box/index.html) | ✅ | ❌ | ❌ |
### 加载器特性
| 源 | 文档延迟加载 | 异步支持
| :---: | :---: | :---: |
| BoxLoader | ✅ | ❌ |
| BoxBlobLoader | ✅ | ❌ |

## 设置

为了使用 Box 包，您需要准备以下几项：

* 一个 Box 账户 —如果您目前不是 Box 用户，或者想在生产环境之外测试您的 Box 实例，您可以使用 [免费开发者账户](https://account.box.com/signup/n/developer#ty9l3)。
* [一个 Box 应用](https://developer.box.com/guides/getting-started/first-application/) — 它在 [开发者控制台](https://account.box.com/developers/console) 中配置，并且对于 Box AI，必须启用 `Manage AI` 范围。您还可以在此处选择您的身份验证方法。
* 该应用必须由 [管理员启用](https://developer.box.com/guides/authorization/custom-app-approval/#manual-approval)。对于免费开发者账户，启用者是注册该账户的人员。

### 凭据

在这些示例中，我们将使用 [令牌身份验证](https://developer.box.com/guides/authentication/tokens/developer-tokens)。它可以与任何 [身份验证方法](https://developer.box.com/guides/authentication/) 一起使用。只需使用您选择的方法获取令牌。如果您想了解更多关于如何将其他身份验证类型与 `langchain-box` 结合使用，请访问 [Box 提供商](/docs/integrations/providers/box) 文档。

In [1]:
import getpass
import os

box_developer_token = getpass.getpass("Enter your Box Developer Token: ")

Enter your Box Developer Token:  ········


要启用对模型调用的自动跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

安装 **langchain_box**。

In [ ]:
%pip install -qU langchain_box

## 初始化

### 加载文件

如果您想加载文件，必须在实例化时提供文件 ID 的 `List`。

这需要 1 条信息：

*   **box\_file\_ids** (`List[str]`) - Box 文件 ID 列表。

#### BoxLoader

In [2]:
from langchain_box.document_loaders import BoxLoader

box_file_ids = ["1514555423624", "1514553902288"]

loader = BoxLoader(
    box_developer_token=box_developer_token,
    box_file_ids=box_file_ids,
    character_limit=10000,  # Optional. Defaults to no limit
)

#### BoxBlobLoader

In [4]:
from langchain_box.blob_loaders import BoxBlobLoader

box_file_ids = ["1514555423624", "1514553902288"]

loader = BoxBlobLoader(
    box_developer_token=box_developer_token, box_file_ids=box_file_ids
)

### 从文件夹加载

如果你希望从文件夹加载文件，则必须在实例化时提供一个包含 Box 文件夹 ID 的 `str`。

这需要 1 条信息：

*   **box_folder_id** (`str`) - 一个包含 Box 文件夹 ID 的字符串。

#### BoxLoader

In [ ]:
from langchain_box.document_loaders import BoxLoader

box_folder_id = "260932470532"

loader = BoxLoader(
    box_folder_id=box_folder_id,
    recursive=False,  # Optional. return entire tree, defaults to False
    character_limit=10000,  # Optional. Defaults to no limit
)

#### BoxBlobLoader

In [ ]:
from langchain_box.blob_loaders import BoxBlobLoader

box_folder_id = "260932470532"

loader = BoxBlobLoader(
    box_folder_id=box_folder_id,
    recursive=False,  # Optional. return entire tree, defaults to False
)

### 使用 BoxBlobLoader 搜索文件

如果您需要搜索文件，`BoxBlobLoader` 提供了两种方法。首先，您可以执行全文搜索，并可选择提供搜索选项来缩小搜索范围。

这需要以下 1 个信息：

*   **query** (`str`) - 包含要执行的搜索查询的字符串。

您还可以提供一个 `BoxSearchOptions` 对象来缩小搜索范围
*   **box_search_options** (`BoxSearchOptions`)

#### BoxBlobLoader 搜索

In [2]:
from langchain_box.blob_loaders import BoxBlobLoader
from langchain_box.utilities import BoxSearchOptions, DocumentFiles, SearchTypeFilter

box_folder_id = "260932470532"

box_search_options = BoxSearchOptions(
    ancestor_folder_ids=[box_folder_id],
    search_type_filter=[SearchTypeFilter.FILE_CONTENT],
    created_date_range=["2023-01-01T00:00:00-07:00", "2024-08-01T00:00:00-07:00,"],
    file_extensions=[DocumentFiles.DOCX, DocumentFiles.PDF],
    k=200,
    size_range=[1, 1000000],
    updated_data_range=None,
)

loader = BoxBlobLoader(
    box_developer_token=box_developer_token,
    query="Victor",
    box_search_options=box_search_options,
)

您还可以根据 Box 元数据搜索内容。如果您的 Box 实例使用元数据，则可以搜索任何具有特定元数据模板并满足特定标准的文档，例如查找上个季度创建的、总金额大于或等于 500 美元的发票。

这需要 1 条信息：

*   **query** (`str`) - 包含要执行的搜索查询的字符串。

您还可以提供 `BoxSearchOptions` 对象来缩小搜索范围
*   **box_search_options** (`BoxSearchOptions`)

#### BoxBlobLoader 元数据查询

In [ ]:
from langchain_box.blob_loaders import BoxBlobLoader
from langchain_box.utilities import BoxMetadataQuery

query = BoxMetadataQuery(
    template_key="enterprise_1234.myTemplate",
    query="total >= :value",
    query_params={"value": 100},
    ancestor_folder_id="260932470532",
)

loader = BoxBlobLoader(box_metadata_query=query)

## 加载

#### BoxLoader

In [3]:
docs = loader.load()
docs[0]

Document(metadata={'source': 'https://dl.boxcloud.com/api/2.0/internal_files/1514555423624/versions/1663171610024/representations/extracted_text/content/', 'title': 'Invoice-A5555_txt'}, page_content='Vendor: AstroTech Solutions\nInvoice Number: A5555\n\nLine Items:\n    - Gravitational Wave Detector Kit: $800\n    - Exoplanet Terrarium: $120\nTotal: $920')

In [4]:
print(docs[0].metadata)

{'source': 'https://dl.boxcloud.com/api/2.0/internal_files/1514555423624/versions/1663171610024/representations/extracted_text/content/', 'title': 'Invoice-A5555_txt'}


#### BoxBlobLoader

In [7]:
for blob in loader.yield_blobs():
    print(f"Blob({blob})")

Blob(id='1514555423624' metadata={'source': 'https://app.box.com/0/260935730128/260931903795/Invoice-A5555.txt', 'name': 'Invoice-A5555.txt', 'file_size': 150} data="b'Vendor: AstroTech Solutions\\nInvoice Number: A5555\\n\\nLine Items:\\n    - Gravitational Wave Detector Kit: $800\\n    - Exoplanet Terrarium: $120\\nTotal: $920'" mimetype='text/plain' path='https://app.box.com/0/260935730128/260931903795/Invoice-A5555.txt')
Blob(id='1514553902288' metadata={'source': 'https://app.box.com/0/260935730128/260931903795/Invoice-B1234.txt', 'name': 'Invoice-B1234.txt', 'file_size': 168} data="b'Vendor: Galactic Gizmos Inc.\\nInvoice Number: B1234\\nPurchase Order Number: 001\\nLine Items:\\n    - Quantum Flux Capacitor: $500\\n    - Anti-Gravity Pen Set: $75\\nTotal: $575'" mimetype='text/plain' path='https://app.box.com/0/260935730128/260931903795/Invoice-B1234.txt')


## 懒加载

#### 仅 BoxLoader

In [ ]:
page = []
for doc in loader.lazy_load():
    page.append(doc)
    if len(page) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        page = []

## 额外字段

所有 Box 连接器都提供了从 Box `FileFull` 对象中选择额外字段的能力，以便作为自定义 LangChain 元数据返回。每个对象都接受一个可选的名为 `extra_fields` 的 `List[str]`，其中包含返回对象中的 json 键，例如 `extra_fields=["shared_link"]`。

连接器会将此字段添加到集成运行所需的字段列表中，然后将结果添加到在 `Document` 或 `Blob` 中返回的元数据里，例如 `"metadata" : { "source" : "source, "shared_link" : "shared_link" }`。如果该字段对于特定文件不可用，它将被返回为空字符串，例如 `"shared_link" : ""`.

## API 参考

有关 BoxLoader 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/api_reference/box/document_loaders/langchain_box.document_loaders.box.BoxLoader.html)

## 帮助

如果您有任何疑问，可以查看我们的 [开发者文档](https://developer.box.com) 或在我们的 [开发者社区](https://community.box.com) 中与我们联系。